In [124]:
import os
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop

In [125]:
# Cada uno aqui ubicad vuestra carpeta de datos
data_dir = os.path.join("..","..","..","data")

In [126]:
train_data, train_labels, test_data, test_labels, test_names = load_data(data_dir, norm=False)


In [127]:
select_train_data = np.array([train_data[i] for i, label in enumerate(train_labels) if label in (21,22)])
select_train_labels = np.array([label for label in train_labels if label in (21,22)])
select_train_labels = tf.one_hot(select_train_labels, depth=2)

In [128]:
pca_transformer = PCA().fit(select_train_data.reshape(-1,(224*224*3)))
pca_train_data = pca_transformer.transform(select_train_data.reshape(-1,(224*224*3)))

In [131]:
model_path = os.path.join(os.getcwd(), "pca+nn.h5")
if os.path.exists(model_path):
    model = load_model(model_path)
else:
    inputs = Input(shape=(pca_train_data.shape[1]))
    x = Dense(256, activation='relu')(inputs)
    x = Dense(128, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(16, activation='relu')(x)

    outputs = Dense(2, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=RMSprop(0.0001),loss="binary_crossentropy", metrics=['accuracy'])
    model.fit(pca_train_data, select_train_labels, epochs=10, batch_size=16, validation_split=0.2)

In [132]:
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 708)]             0         
                                                                 
 dense_66 (Dense)            (None, 128)               90752     
                                                                 
 dense_67 (Dense)            (None, 64)                8256      
                                                                 
 dense_68 (Dense)            (None, 16)                1040      
                                                                 
 dense_69 (Dense)            (None, 2)                 34        
                                                                 
Total params: 100,082
Trainable params: 100,082
Non-trainable params: 0
_________________________________________________________________


In [ ]:
select_test_data = np.array([test_data[i] for i, label in enumerate(test_labels) if label in (21,22)])
pca_test_data = pca_transformer.transform(select_test_data.reshape(-1,(224*224*3)))

select_test_labels = np.array([label for label in test_labels if label in (21,22)])
select_test_labels = tf.one_hot(select_test_labels, depth=2)

In [ ]:
model.evaluate(pca_test_data, select_test_labels)

4/4 [==============================] - 0s 4ms/step - loss: 6.1164e-24 - accuracy: 0.9612


[6.116373075202318e-24, 0.9611650705337524]

In [ ]:
# Cuidado descomentando esto
#model.save_weights("pca+nn_weights.h5")
#model.save("pca+nn.h5")